In [392]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np

In [393]:
start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime(2022, 12, 31)
ticker = "SPY"
NEUTRAL_BENCHMARK = 0.005

In [394]:
train_data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [395]:
print(type(train_data))

<class 'pandas.core.frame.DataFrame'>


In [396]:
train_data["daily_return"] = train_data["Adj Close"].pct_change()
train_data["state"] = np.select(
    [train_data["daily_return"] > NEUTRAL_BENCHMARK, train_data["daily_return"] < -1 * NEUTRAL_BENCHMARK],
    ["up", "down"],
    default="neutral"
)
train_data.head(15)

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2021-01-04,375.309998,375.450012,364.820007,368.790009,354.197418,110210800,NaN,neutral
2021-01-05,368.100006,372.500000,368.049988,371.329987,356.636841,66426200,0.006887,up
2021-01-06,369.709991,376.980011,369.119995,373.549988,358.769073,107997700,0.005979,up
2021-01-07,376.100006,379.899994,375.910004,379.100006,364.099457,68766800,0.014857,up
2021-01-08,380.589996,381.489990,377.100006,381.260010,366.174011,71677200,0.005698,up
2021-01-11,377.850006,380.579987,377.720001,378.690002,363.705658,51034700,-0.006741,down
2021-01-12,378.890015,379.859985,376.359985,378.769989,363.782471,52547700,0.000211,neutral
2021-01-13,378.690002,380.859985,377.850006,379.790009,364.762115,45303600,0.002693,neutral
2021-01-14,380.589996,381.130005,378.100006,378.459991,363.484833,49989100,-0.003502,neutral


In [397]:
up_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))

down_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))

neutral_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))

In [398]:
transition_matrix = pd.DataFrame({
    "up": [up_to_up, up_to_neutral, up_to_down],
    "neutral": [neutral_to_up, neutral_to_neutral, neutral_to_down],
    "down": [down_to_up, down_to_neutral, down_to_down]
}, index=["up", "neutral", "down"])
transition_matrix.head()
print(transition_matrix)

               up   neutral      down
up       0.314103  0.313433  0.301370
neutral  0.384615  0.462687  0.321918
down     0.301282  0.218905  0.376712


In [399]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.152657,135480400,NaN,down
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.441467,146322800,-0.009722,down
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.018677,118387200,0.007968,up
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.675278,108872000,-0.004703,down
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.844353,132079200,0.016086,up


In [400]:
# Test Predictions
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 6, 1)
ticker = "AAPL"
test_data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [401]:
from backtesting import Backtest, Strategy

In [402]:
from enum import Enum

class Trend(Enum):
    DOWN = "Down"
    UP = "Up"
    NEUTRAL = "Neutral"

In [403]:
max_up_values = max(up_to_up, down_to_up, neutral_to_up)
if max_up_values == up_to_up:
    trend_up = Trend.UP
elif max_up_values == down_to_up:
    trend_up = Trend.DOWN
else:
    trend_up = Trend.NEUTRAL

# Determine trend.down
max_down_values = max(up_to_down, down_to_down, neutral_to_down)
if max_down_values == up_to_down:
    trend_down = Trend.UP
elif max_down_values == down_to_down:
    trend_down = Trend.DOWN
else:
    trend_down = Trend.NEUTRAL

In [404]:
class MarkovStrategy(Strategy):
    def init(self):
        """init"""

    def get_state(self):
        close_yest = self.data.Close[-2]
        close_today = self.data.Close[-1]

        pct = (close_today - close_yest) / close_yest

        if pct > NEUTRAL_BENCHMARK:
            return Trend.UP
        elif pct < -NEUTRAL_BENCHMARK:
            return Trend.DOWN
        else:
            return Trend.NEUTRAL


    def next(self):
        state = self.get_state()

        self.position.close()
        
        if state == trend_down:
            self.sell()
        elif state == trend_up:
            self.buy()

        

In [405]:
bt = Backtest(train_data, MarkovStrategy, cash=10000, commission=0)
stats = bt.run()
stats

Start                     2021-01-04 00:00:00
End                       2022-12-30 00:00:00
Duration                    725 days 00:00:00
Exposure Time [%]                    81.11332
Equity Final [$]                  9668.091675
Equity Peak [$]                  11244.693176
Return [%]                          -3.319083
Buy & Hold Return [%]                3.698577
Return (Ann.) [%]                   -1.676844
Volatility (Ann.) [%]               15.551901
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -14.643363
Avg. Drawdown [%]                   -6.693558
Max. Drawdown Duration      388 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                                  303
Win Rate [%]                        54.125413
Best Trade [%]                       5.001621
Worst Trade [%]                     -4.522893
Avg. Trade [%]                    

In [406]:
bt.plot()

/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p7654', ...)

In [407]:
bt